In [ ]:
import glob
import os
from tqdm import tqdm
import pandas as pd
import numpy as np
import torch
import pickle

from transformers import AutoTokenizer, AutoModel, AutoConfig

batch_size = 128
device = "cuda:6"

Translation done in translation.ipynb

English NER done in nlstruct_predict.ipynb

Normalization done in deep_mlg_inference_script.ipynb

results saved in a pkl file

In [ ]:
# read results
gold_file_transl = '/export/home/cse200093/deep_mlg_normalization/df_mentions_EMEA_test_en_4_types.pkl'
open_file = open(gold_file_transl, "rb")
df_res = pickle.load(open_file)
open_file.close()

df_res.rename(columns={'cui_res':'cuis'},inplace=True)
#df_res = df_res[(df_res['label']=='PROC')|(df_res['label']=='DEVI')|(df_res['label']=='DISO')|(df_res['label']=='CHEM')]
df_res

In [ ]:
# gold standard path
# stand_path = '/export/home/cse200093/brat_data/Expe_TRANSL/expe_translation_Gold_after_conversion_train_test/test/'
stand_path = '/export/home/cse200093/brat_data/QUAERO_FrenchMed/corpus/test/EMEA'

In [ ]:
# select all ann files
os.chdir(stand_path)
my_files = glob.glob('*.ann')
len(my_files)

In [ ]:
phrases = []
cuis = []
types = []
dic = {} # necessary when len(phrases) != len(cuis)
for file in tqdm(my_files):
    phrase = None
    code = None
    type_name = None
    f = open(stand_path+'/'+file, "r")
    for line in f:
        line = line.rstrip('\n')
        if line.startswith('T'):
            type_name = line.split('\t')[1].split(' ')[0]
            phrase = line.split('\t')[2]
            code = line.split('\t')[0] # necessary when len(phrases) != len(cuis)
            dic[file+'.'+phrase+'.'+code] = 'no cui' # necessary when len(phrases) != len(cuis)
            #phrases.append(phrase)
            #types.append(type_name)
        elif line.startswith('#'):
            cui = line.split('\t')[-1]
            dic[file+'.'+phrase+'.'+code] = (cui,type_name) # necessary when len(phrases) != len(cuis)
            #cui = cui.split(',')[0][1:].rstrip('\"') # necessary for Mantra
            #cuis.append(cui)

In [ ]:
# necessary when len(phrases) != len(cuis)
print(len(dic))
no_cuis = []
files = []
# phrases_eng = []
dic_file_ent = {}
for key in tqdm(dic):
    if dic[key] != 'no cui':
        phrases.append(key.split('.')[2])
        # necessary with translation
        # phrases_eng.append(translator.translate(key.split('.')[2]).text)
        cuis.append(dic[key][0])
        types.append(dic[key][1])
        file = key.split('.')[0]
        files.append(file)
        if file in dic_file_ent.keys():
            dic_file_ent[file].append(key.split('.')[2])
        else:
            dic_file_ent[file] = [key.split('.')[2]]
    else:
        #print(key)
        no_cuis.append(key)
        
print(len(cuis),len(phrases),len(types),len(files),len(dic_file_ent))

In [ ]:
df_stand = pd.DataFrame({'doc_id':files, 'phrases':phrases, 'cuis':cuis, 'types':types})
df_stand = df_stand[(df_stand['types']=='PROC')|(df_stand['types']=='DEVI')|(df_stand['types']=='DISO')|(df_stand['types']=='CHEM')]
df_stand

In [ ]:
def get_dic(df):
    dic = {}
    docs = list(df['doc_id'])
    for doc in docs:
        cuis = list(df[df['doc_id']==doc]['cuis'])
        dic[doc] = cuis
    return dic

In [ ]:
def convert_list_to_dic(dic):
    for file_name in tqdm(dic):
        key_list = list(set(dic[file_name]))
        value_list = [dic[file_name].count(x) for x in key_list]
        res = dict(zip(key_list, value_list))
        dic[file_name] = res

In [ ]:
dic_stand = get_dic(df_stand)
dic_res = get_dic(df_res)

convert_list_to_dic(dic_stand)
convert_list_to_dic(dic_res)

In [ ]:
df_compare = pd.DataFrame(columns = ['file_name','correct_ents','pred_ents','text'])

for file in tqdm(dic_stand):
    df_compare = df_compare.append({'file_name':file, 'correct_ents':dic_stand[file], 
                       'pred_ents':dic_res[file], 'text':dic_file_ent[file]},
                     ignore_index=True)
    
df_compare

In [ ]:
df_compare['built_TP'] = 0
df_compare['built_FP'] = 0
df_compare['built_FN'] = 0

df_compare['built_precision'] = 0
df_compare['built_recall'] = 0
df_compare['built_f1-score'] = 0

# calculate TP,FP,FN for each model
for index, row in tqdm(df_compare.iterrows()):
    for cui in row['pred_ents']:
        if cui in row['correct_ents']:
            df_compare.loc[index,'built_TP'] += min(row['correct_ents'][cui], row['pred_ents'][cui])
        else:
            df_compare.loc[index,'built_FP'] += row['pred_ents'][cui]
    for cui in row['correct_ents']:
        if cui not in row['pred_ents']:
            df_compare.loc[index,'built_FN'] += row['correct_ents'][cui]
            
            
# calculate precision and recall for each model
for index, row in tqdm(df_compare.iterrows()):
    if row['built_TP'] != 0:
        df_compare.loc[index,'built_precision'] = row['built_TP']/(row['built_TP']+row['built_FP'])

    if row['built_TP'] != 0:
        df_compare.loc[index,'built_recall'] = row['built_TP']/(row['built_TP']+row['built_FN'])

# calculate f1-score for each model
for index, row in tqdm(df_compare.iterrows()):
    if row['built_TP'] != 0:
        df_compare.loc[index,'built_f1-score'] = (2*row['built_precision']*row['built_recall'])/(row['built_precision']+row['built_recall'])
            

df_compare

In [ ]:
# calculate mean precision, mean recall, mean f1-score

built_avg_precision = np.mean(df_compare['built_precision'])
built_avg_recall = np.mean(df_compare['built_recall'])
built_avg_f1_score = np.mean(df_compare['built_f1-score'])

print(f'Average precision of the built model: {built_avg_precision}')
print(f'Average recall of the built model: {built_avg_recall}')
print(f'Average f1-score of the built model: {built_avg_f1_score}')

In [ ]:
# save df_compare for boostrap.ipynb
df_compare.to_pickle('/export/home/cse200093/Expe_Translation/Axes_script/EMEA_res_axis2.pkl')